In [8]:
!pip install numpy pandas matplotlib seaborn scikit-learn torch torchvision



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Use MPS (Apple GPU) if available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)


Using device: mps


In [11]:
# Transform: Convert images to tensors
transform = transforms.ToTensor()

# Load MNIST dataset
train_data = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_data = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1000)

In [12]:
# Define a simple neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten image
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [13]:
# Initialize model, loss, optimizer
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Train for 1 epoch (for demo)
for epoch in range(1):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}]  Loss: {loss.item():.4f}")

Train Epoch: 0 [0/60000]  Loss: 2.3055
Train Epoch: 0 [6400/60000]  Loss: 0.2906
Train Epoch: 0 [12800/60000]  Loss: 0.4363
Train Epoch: 0 [19200/60000]  Loss: 0.2642
Train Epoch: 0 [25600/60000]  Loss: 0.1970
Train Epoch: 0 [32000/60000]  Loss: 0.1709
Train Epoch: 0 [38400/60000]  Loss: 0.3275
Train Epoch: 0 [44800/60000]  Loss: 0.1078
Train Epoch: 0 [51200/60000]  Loss: 0.1222
Train Epoch: 0 [57600/60000]  Loss: 0.1258


In [14]:
# Evaluate model
model.eval()
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()

accuracy = 100. * correct / len(test_loader.dataset)
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 94.61%
